In [87]:
# now we will create a master dataset that combines all three previous datasets (noaa, nasa, epa) 
# we can tweak it to optimize it for ML and predictive modeling. 
# will join on year
import pandas as pd

In [70]:
# Load historical NOAA data (1975-2024)
noaa_df = pd.read_csv("../data/Cleaned/Master_NOAA_1975_2024.csv")

# Load NASA (temperature anomaly) and EPA (GHG emissions) data
nasa_df = pd.read_csv("../data/Cleaned/Master_NASA_1974_2024.csv")
epa_df = pd.read_csv("../data/Cleaned/Master_Emissions.csv")

In [85]:
noaa_df.head()

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,TOTAL_DAMAGE,TOTAL_DEATHS,TOTAL_INJURIES
0,201408,26,2000,201408,26,2300,89834.0,541665,ILLINOIS,17.0,...,1.0,ESE,NIANTIC,39.8967,-89.1125,An approaching cold front triggered strong to ...,A severe thunderstorm produced 2.00 to 3.00 of...,2.000000e+07,0.0,0.0
1,201408,26,600,201408,27,2300,89017.0,536430,CALIFORNIA,6.0,...,NaN,NaN,NaN,NaN,NaN,"Hurricane Marie, which reached Category 5 stat...",Large southeast swells from Hurricane Marie ge...,4.000000e+07,0.0,1.0
2,201412,1,0,201412,31,0,91468.0,554395,CALIFORNIA,6.0,...,NaN,NaN,NaN,NaN,NaN,"On January 17th, 2014, Governor Jerry Brown de...",Impacts to the region included: Reservoirs dro...,1.500000e+09,0.0,0.0
3,201412,23,1420,201412,23,1436,91483.0,552175,MISSISSIPPI,28.0,...,10.0,E,COLUMBIA MARION ARPT,31.2273,-89.8331,"During the afternoon of Dec 23, just enough in...",This tornado first touched down just east of t...,2.500000e+07,3.0,50.0
4,201401,10,700,201401,10,1030,80592.0,493037,NEW JERSEY,34.0,...,NaN,NaN,NaN,NaN,NaN,A winter weather event dropped snow across nor...,NaN,0.000000e+00,1.0,115.0


In [89]:
import pandas as pd

# Load historical NOAA data (1975-2024)
noaa_df = pd.read_csv("../data/Cleaned/Master_NOAA_1975_2024.csv")

# Load NASA (temperature anomaly) and EPA (GHG emissions) data
nasa_df = pd.read_csv("../data/Cleaned/Master_NASA_1974_2024.csv")
epa_df = pd.read_csv("../data/Cleaned/Master_Emissions.csv")

# Extract YEAR from NOAA dataset
noaa_df["YEAR"] = noaa_df["BEGIN_YEARMONTH"].astype(str).str[:4].astype(int)

# Convert YEAR columns to match
nasa_df["YEAR"] = nasa_df["Year"].astype(int)
epa_df["YEAR"] = epa_df["Year"].astype(int)

# Drop redundant columns
nasa_df.drop(columns=["Year"], inplace=True)
epa_df.drop(columns=["Year"], inplace=True)

# Merge NOAA with NASA and EPA data
historical_df = noaa_df.merge(nasa_df, on="YEAR", how="left")
historical_df = historical_df.merge(epa_df, on="YEAR", how="left")

# Select relevant columns for Tableau
historical_df = historical_df[[
    "YEAR", "STATE", "EVENT_TYPE", "TOTAL_INJURIES", "TOTAL_DEATHS", "TOTAL_DAMAGE",
    "Per_Capita_Emissions", "NASA_Global_Temp_Anomaly_C",
    "BEGIN_LAT", "BEGIN_LON"  # Keep Latitude & Longitude for mapping
]]




In [107]:
historical_df.head()

,YEAR,STATE,EVENT_TYPE,TOTAL_INJURIES,TOTAL_DEATHS,TOTAL_DAMAGE,Per_Capita_Emissions,NASA_Global_Temp_Anomaly_C,BEGIN_LAT,BEGIN_LON
0,2014,ILLINOIS,Flash Flood,0.0,0.0,2.000000e+07,19.926178,0.75,39.8967,-89.1125
1,2014,CALIFORNIA,High Surf,1.0,0.0,4.000000e+07,19.926178,0.75,NaN,NaN
2,2014,CALIFORNIA,Drought,0.0,0.0,1.500000e+09,19.926178,0.75,NaN,NaN
3,2014,MISSISSIPPI,Tornado,50.0,3.0,2.500000e+07,19.926178,0.75,31.2273,-89.8331
4,2014,NEW JERSEY,Winter Weather,115.0,1.0,0.000000e+00,19.926178,0.75,NaN,NaN


In [125]:
print(sorted(historical_df['YEAR'].unique()))

[1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]


In [109]:
#we have missing values in our lat and lon columns. We want to imput these values based on mode (most common place of disaster), and our fallback will be mean (if no most common place is found).

In [113]:
import pandas as pd

# Load historical NOAA data (1975-2024)
noaa_df = pd.read_csv("../data/Cleaned/Master_NOAA_1975_2024.csv")

# Load NASA (temperature anomaly) and EPA (GHG emissions) data
nasa_df = pd.read_csv("../data/Cleaned/Master_NASA_1974_2024.csv")
epa_df = pd.read_csv("../data/Cleaned/Master_Emissions.csv")

# Extract YEAR from NOAA dataset
noaa_df["YEAR"] = noaa_df["BEGIN_YEARMONTH"].astype(str).str[:4].astype(int)

# Convert YEAR columns to match for merging
nasa_df["YEAR"] = nasa_df["Year"].astype(int)
epa_df["YEAR"] = epa_df["Year"].astype(int)

# Drop redundant 'Year' columns
nasa_df.drop(columns=["Year"], inplace=True)
epa_df.drop(columns=["Year"], inplace=True)

# Step 1: Compute Mode & Mean Lat/Lon Per State
state_latlon_modes = noaa_df.groupby("STATE")[["BEGIN_LAT", "BEGIN_LON"]].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
state_latlon_means = noaa_df.groupby("STATE")[["BEGIN_LAT", "BEGIN_LON"]].mean()

# Step 2: Fill Missing Lat/Lon Using Mode, then Mean
def fill_missing_latlon(row):
    if pd.isna(row["BEGIN_LAT"]) or pd.isna(row["BEGIN_LON"]):
        state = row["STATE"]
        if state in state_latlon_modes.index and not pd.isna(state_latlon_modes.loc[state, "BEGIN_LAT"]):
            row["BEGIN_LAT"] = state_latlon_modes.loc[state, "BEGIN_LAT"]
            row["BEGIN_LON"] = state_latlon_modes.loc[state, "BEGIN_LON"]
        elif state in state_latlon_means.index:
            row["BEGIN_LAT"] = state_latlon_means.loc[state, "BEGIN_LAT"]
            row["BEGIN_LON"] = state_latlon_means.loc[state, "BEGIN_LON"]
    return row

# Apply imputation function
noaa_df = noaa_df.apply(fill_missing_latlon, axis=1)

# Merge NOAA with NASA and EPA data
historical_df = noaa_df.merge(nasa_df, on="YEAR", how="left")
historical_df = historical_df.merge(epa_df, on="YEAR", how="left")

# Select relevant columns for Tableau
historical_df = historical_df[[
    "YEAR", "STATE", "EVENT_TYPE", "TOTAL_INJURIES", "TOTAL_DEATHS", "TOTAL_DAMAGE",
    "Per_Capita_Emissions", "NASA_Global_Temp_Anomaly_C",
    "BEGIN_LAT", "BEGIN_LON"  # Keep Latitude & Longitude for mapping
]]

# Save cleaned dataset
historical_df.to_csv("historical_disaster_data.csv", index=False, header=True)


In [117]:
historical_df.head()

,YEAR,STATE,EVENT_TYPE,TOTAL_INJURIES,TOTAL_DEATHS,TOTAL_DAMAGE,Per_Capita_Emissions,NASA_Global_Temp_Anomaly_C,BEGIN_LAT,BEGIN_LON
0,2014,ILLINOIS,Flash Flood,0.0,0.0,2.000000e+07,19.926178,0.75,39.8967,-89.11250
1,2014,CALIFORNIA,High Surf,1.0,0.0,4.000000e+07,19.926178,0.75,34.4421,-122.88333
2,2014,CALIFORNIA,Drought,0.0,0.0,1.500000e+09,19.926178,0.75,34.4421,-122.88333
3,2014,MISSISSIPPI,Tornado,50.0,3.0,2.500000e+07,19.926178,0.75,31.2273,-89.83310
4,2014,NEW JERSEY,Winter Weather,115.0,1.0,0.000000e+00,19.926178,0.75,39.5000,-75.42000


In [127]:
print(sorted(historical_df['YEAR'].unique()))

[1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
